### Análise de falhas em amostra de voos de uma aeronave

O notebook apresenta uma análise relativa ao disparo de falhas no Sistema de Bleed de uma aeronave. Na análise abaixo, somente foram considerados arquivos cuja aeronave apresentou todas as fases de voo. O objetivo é identificar se existe alguma relação entre a fase de voo e o disparo de falhas no Sistema de Bleed, visto que, inicialmente, as falhas parecem estar concentradas em arquivos cuja aeronave apresenta uma única fase de voo.

In [129]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [130]:
df = None
folder = "./data"
archives = [arq for arq in os.listdir(folder) if arq.endswith('.parquet')]
columns = ['message0418DAA-1', 'message0422DAA-1', 'phaseOfFlight-1']

Abaixo, encontra-se a lógica utilizada para filtrar os arquivos que compõem o novo dataframe. São considerados os seguintes atributos: as variáveis-alvo ('message0418DAA-1' e 'message0418DAA-2') e a fase de voo ('flight_phase').

In [131]:
contador = 0
for archive in archives:
    voo_normal = True
    path_archive = os.path.join(folder, archive)
    df_temp = pd.read_parquet(path_archive, columns=columns)
    df_temp['flight'] = archive
    df_temp['message0418DAA-1'] = df_temp['message0418DAA-1'].astype('Int8')
    df_temp['message0422DAA-1'] = df_temp['message0422DAA-1'].astype('Int8')
    df_temp['phaseOfFlight-1'] = df_temp['phaseOfFlight-1'].astype('Int8')
    df_temp.fillna(method='ffill', inplace=True)
    df_temp.fillna(method='bfill', inplace=True)

    if df is None:
        df = df_temp
    else:
        df = pd.concat([df, df_temp])
    df_temp = None
    contador += 1
    if contador % 163 == 0:
        break

Posteriormente, é adotado o método Forward Fill, a fim de preencher os valores nulos com o último valor válido entre os registros existentes. Essa decisão justifica-se pelo fato de que, embora alguns registros apresentassem valor nulo em determinado atributo, os demais atributos apresentavam valores válidos.

In [132]:
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
df

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
0,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
1,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
2,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
3,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
4,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
...,...,...,...,...
23056,0,0,0,TCRF_ARCHIVE_06120018_20220618231001.parquet
23057,0,0,0,TCRF_ARCHIVE_06120018_20220618231001.parquet
23058,0,0,0,TCRF_ARCHIVE_06120018_20220618231001.parquet
23059,0,0,0,TCRF_ARCHIVE_06120018_20220618231001.parquet


In [133]:
df["flight"].nunique()

163

In [134]:
df["phaseOfFlight-1"].value_counts()

phaseOfFlight-1
1    8012800
0    6882133
2    4677446
Name: count, dtype: Int64

In [135]:
df.head()

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
0,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
1,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
2,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
3,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet
4,0,0,0,TCRF_ARCHIVE_06120018_20220523142237.parquet


In [136]:
df.isna().sum()

message0418DAA-1    0
message0422DAA-1    0
phaseOfFlight-1     0
flight              0
dtype: int64

Abaixo, é realizada a contagem de disparos de mensagem de erro no dataset formado. Além disso, ocorre uma comparação entre cada variável-alvo e entre cada valor que tais atributos podem apresentar. No caso das variáveis-alvo, foram identificados os valores '0', '5' e '7', e suas quantidades variam a depender de cada variável.

In [137]:
df["message0422DAA-1"].value_counts()

message0422DAA-1
0    19572379
Name: count, dtype: Int64

In [138]:
df["message0418DAA-1"].value_counts()

message0418DAA-1
0    19568258
7        3921
5         200
Name: count, dtype: Int64

In [139]:
df[df['message0422DAA-1'] == 7]

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight


In [140]:
df[df['message0422DAA-1'] == 5]

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
